<a href="https://colab.research.google.com/github/Frantisek21/Final_project/blob/main/Usov_Suran_FE_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Homework 2
## Financial Econometrics I
#### *Artem Usov*, *František Šuráň*

In [2]:
# Package handling
packages <- c("dplyr", "quantmod", "tseries","rugarch","ggplot2","gridExtra")

# Package installation
installed_packages <- packages %in% rownames(installed.packages())
if (any(installed_packages == FALSE)) {
  install.packages(packages[!installed_packages])
  print(paste("Installing package ", packages[!installed_packages],"...", sep = ""))
}

# Packages loading
invisible(lapply(packages, library, character.only = TRUE))
rm(list = ls()) #Clean environment

# Settings
set.seed(26830192)
options(warn=-1, repr.plot.width = 14, repr.plot.height = 4,"getSymbols.warning4.0"=FALSE,scipen=100)

## Data

In [7]:
stocks<-read.csv("symbols2.csv")
numbers<-sample(1:nrow(stocks), 100, replace=F) %>% sort()
# Random numbers from this seed
numbers
stocks<-stocks[numbers,] %>% select(2) %>% pull() %>% as.character()

[1]   2   3   7   9  11  12  13  22  31  39  40  44  45  54  64  66  71  75
 [19]  85  91  92  95  97 103 108 110 111 113 115 118 128 137 142 147 152 159
 [37] 160 170 172 183 192 193 194 196 197 198 201 205 213 215 220 221 223 224
 [55] 231 234 237 239 240 241 242 244 254 255 258 259 269 272 275 277 278 283
 [73] 284 287 292 293 304 306 307 311 315 318 319 322 332 338 339 340 342 352
 [91] 356 358 361 362 364 366 369 370 375 377

In [4]:
stocks

[1] "ACE"  "AFL"  "A"    "ATI"  "AEE"  "AIG"  "AMP"  "APH"  "AIV"  "AIZ" 
 [11] "AZO"  "BDX"  "BBBY" "BA"   "BXP"  "KMX"  "CI"   "CSCO" "CMS"  "CTSH"
 [21] "CMA"  "STZ"  "CSX"  "CVS"  "DRI"  "DLTR" "RRD"  "EIX"  "EA"   "EQT" 
 [31] "EFX"  "EQR"  "EXPD" "FIS"  "FSLR" "GME"  "GCI"  "GD"   "GT"   "GWW" 
 [41] "HIG"  "HAS"  "HPQ"  "HD"   "HRL"  "IGT"  "IP"   "IVZ"  "JCI"  "LEG" 
 [51] "LLY"  "L"    "MAR"  "MAT"  "MCK"  "MET"  "MS"   "MOS"  "MSI"  "NDAQ"
 [61] "NWL"  "NEM"  "NKE"  "JWN"  "NSC"  "NTRS" "NOC"  "NUE"  "OXY"  "ORCL"
 [71] "PLL"  "BTU"  "PBCT" "PXD"  "PNC"  "RL"   "PFG"  "PGR"  "PRU"  "QCOM"
 [81] "RRC"  "RSG"  "ROK"  "STX"  "SPG"  "SWN"  "STT"  "SYY"  "TEL"  "TXN" 
 [91] "TMO"  "TRIP" "UPS"  "VMC"  "WM"   "WAT"  "WDC"  "XEL"  "XRX"  "ZION"

In [8]:
# Getting Stock prices
data_init <- lapply(stocks, function(y){
    tryCatch(
        {
            getSymbols(y,from = "2020-01-01",to="2021-10-31", auto.assign = FALSE)
        },
        error = function(e){
            print(paste("Failed to download", y))
        }
    )
})
names(data_init)<-stocks

[1] "Failed to download BHI"
[1] "Failed to download BLL"
[1] "Failed to download CERN"
[1] "Failed to download RRD"
[1] "Failed to download PBCT"
[1] "Failed to download PKI"


In [9]:
# Selecting the closing price of each stock
data <- lapply(names(data_init), function(y){
    data_init[[y]][, paste0(y, '.Close')]
})
names(data)<-stocks

ERROR: Error in data_init[[y]][, paste0(y, ".Close")]: incorrect number of dimensions


In [11]:
#Zatím jenom chatgpt code
# Read symbols
stocks <- read.csv("symbols2.csv")

# Sample 100 random numbers
set.seed(26830192) # Set seed for reproducibility
numbers <- sample(1:nrow(stocks), 100, replace = FALSE)
selected_stocks <- stocks$Symbol[numbers]

# Download stock data
data_init <- lapply(selected_stocks, function(symbol) {
  tryCatch(
    {
      getSymbols(symbol, from = "2020-01-01", to = "2021-10-31", auto.assign = FALSE)
    },
    error = function(e) {
      print(paste("Failed to download", symbol))
      return(NULL) # Return NULL for failed downloads
    }
  )
})

# Remove NULL elements from data_init
data_init <- data_init[!sapply(data_init, is.null)]

# Select closing prices
data <- lapply(data_init, function(x) Cl(x))

# Optionally, you can check if any symbols failed to download
failed_symbols <- selected_stocks[is.null(data_init)]
if (length(failed_symbols) > 0) {
  cat("Failed to download the following symbols:", paste(failed_symbols, collapse = ", "))
}


[1] "Failed to download RRD"
[1] "Failed to download PBCT"
[1] "Failed to download BBBY"


In [12]:
data

[[1]]
           GME.Close
2020-01-02    1.5775
2020-01-03    1.4700
2020-01-06    1.4625
2020-01-07    1.3800
2020-01-08    1.4300
2020-01-09    1.3875
2020-01-10    1.3575
2020-01-13    1.3575
2020-01-14    1.1775
2020-01-15    1.1525
       ...          
2021-10-18   46.5050
2021-10-19   46.6975
2021-10-20   46.1300
2021-10-21   45.4275
2021-10-22   42.4500
2021-10-25   43.4925
2021-10-26   44.4600
2021-10-27   43.3775
2021-10-28   45.7125
2021-10-29   45.8775

[[2]]
           NKE.Close
2020-01-02    102.20
2020-01-03    101.92
2020-01-06    101.83
2020-01-07    101.78
2020-01-08    101.55
2020-01-09    101.48
2020-01-10    100.90
2020-01-13    102.16
2020-01-14    102.92
2020-01-15    102.79
       ...          
2021-10-18    159.43
2021-10-19    157.82
2021-10-20    158.45
2021-10-21    162.18
2021-10-22    163.48
2021-10-25    164.22
2021-10-26    163.79
2021-10-27    162.35
2021-10-28    164.46
2021-10-29    167.29

[[3]]
           IGT.Close
2020-01-02     15.08
2020-01-03    

In [13]:
non_zero<-sapply(data_init,function(y){
    sum(complete.cases(y))
})
paste0("Full length without NAs: ",max(non_zero))
non_zero[non_zero != max(non_zero)]

[1] "Full length without NAs: 462"

[1] 1

## Problem 1

### 1. Compute the logarithmic returns for the time-series of each stock in your dataset.

In [14]:
# transform to log returns
lret<-lapply(data,function(y){diff(log(y))})
#remove the first row with NA
lret<-lapply(lret,function(y){y[-1, ]})

### 2. For each stock, estimate the parameters of a GARCH(1, 1) model.

In [19]:
# Function GARCH(1,1)
get_fits<-function(y){
    garch_11<-ugarchspec(mean.model = list(armaOrder = c(0,0)),
                      variance.model = list(model = "sGARCH",garchOrder=c(1,1)),
                     distribution.model = "norm")
    fit<-ugarchfit(garch_11,y)
    fit
}

In [20]:
fits<-sapply(lret,get_fits)

ERROR: Error in if (mean(data) == 0) {: missing value where TRUE/FALSE needed


In [21]:
# Function GARCH(1,1)
get_fits <- function(y) {
  # Remove missing values
  y <- na.omit(y)

  # Check if there are still values remaining after removing NAs
  if (length(y) == 0) {
    # If all values were NAs, return NA
    return(NA)
  }

  garch_11 <- ugarchspec(mean.model = list(armaOrder = c(0,0)),
                         variance.model = list(model = "sGARCH", garchOrder = c(1,1)),
                         distribution.model = "norm")
  fit <- try(ugarchfit(garch_11, y), silent = TRUE)
  if(class(fit) == "try-error") {
    # If fitting fails, return NA
    return(NA)
  }
  fit
}

# Apply function to each series of log returns
fits <- lapply(lret, get_fits)

# Filter out NA fits
fits <- fits[!sapply(fits, is.na)]

# Optionally, you can check the number of failed fits
failed_count <- sum(sapply(fits, is.null))
if (failed_count > 0) {
  cat("Failed to fit", failed_count, "series.")
}
